### 特征可视化
一般来说，当一个变量从直觉上来说对所要完成的目标有帮助，就可以将其作为 Feature。至于它是否有效，最简单的方式就是通过图表来直观感受。比如可以使用散点图，箱形图等。

### Dummy Variables
对于Categorical Variable，常用的做法是**one-hot encoding**。即对这一变量创建一组新的伪变量，对应其所有可能的取值。这些变量中只有这条数据对应的取值为 1，其他都为 0。

#### 离散特征取值有大小意义
离散特征的取值有大小的意义时，比如size:[X,XL,XXL],那么就使用数值的映射{X:1,XL:2,XXL:3}

In [4]:
import pandas as pd
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']
])

df.columns = ['color', 'size', 'price', 'class label']

size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1
}
df['size'] = df['size'].map(size_mapping)

class_mapping = {label:idx for idx,label in enumerate(set(df['class label']))}
df['class label'] = df['class label'].map(class_mapping)

df

,color,size,price,class label
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


#### 离散特征的取值无大小意义
离散特征的取值之间没有大小的意义时，比如color：[red,blue],那么就使用one-hot编码。使用**pd.get_dummies()**可以直接创建一组新的伪变量（dummy variables），对应其所有可能的取值。这些变量中只有这条数据对应的取值为 1，其他都为 0。

*注：当变量可能取值的范围很大（比如一共有成百上千类）时，这种简单的方法就不太适用了。*

In [5]:
pd.get_dummies(df)

,size,price,class label,color_blue,color_green,color_red
0,1,10.1,0,0,1,0
1,2,13.5,1,0,0,1
2,3,15.3,0,1,0,0


### Feature Selection
总的来说，我们应该生成尽量多的 Feature，相信 Model 能够挑出最有用的 Feature。但有时先做一遍 Feature Selection 也能带来一些好处：
+ Feature越少，训练越快
+ 有些Feature之间可能存在线性关系，影像Model性能
+ 通过挑选出最重要的Feature，可以将它们之间进行各种运算和操作的结果作为新的Feature，可能带来意外的提高

In [13]:
from sklearn.datasets import load_iris

# 导入IRIS数据集
iris = load_iris()

# 特征矩阵
df_iris = pd.DataFrame(iris.data)
print(df_iris.head())

# 目标向量
df_target = pd.DataFrame(iris.target)
print(df_target.head())

     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
   0
0  0
1  0
2  0
3  0
4  0


In [16]:
# 标准化
from sklearn.preprocessing import StandardScaler
StandardScaler().fit_transform(iris.data)

array([[ -9.00681170e-01,   1.03205722e+00,  -1.34127240e+00,
         -1.31297673e+00],
       [ -1.14301691e+00,  -1.24957601e-01,  -1.34127240e+00,
         -1.31297673e+00],
       [ -1.38535265e+00,   3.37848329e-01,  -1.39813811e+00,
         -1.31297673e+00],
       [ -1.50652052e+00,   1.06445364e-01,  -1.28440670e+00,
         -1.31297673e+00],
       [ -1.02184904e+00,   1.26346019e+00,  -1.34127240e+00,
         -1.31297673e+00],
       [ -5.37177559e-01,   1.95766909e+00,  -1.17067529e+00,
         -1.05003079e+00],
       [ -1.50652052e+00,   8.00654259e-01,  -1.34127240e+00,
         -1.18150376e+00],
       [ -1.02184904e+00,   8.00654259e-01,  -1.28440670e+00,
         -1.31297673e+00],
       [ -1.74885626e+00,  -3.56360566e-01,  -1.34127240e+00,
         -1.31297673e+00],
       [ -1.14301691e+00,   1.06445364e-01,  -1.28440670e+00,
         -1.44444970e+00],
       [ -5.37177559e-01,   1.49486315e+00,  -1.28440670e+00,
         -1.31297673e+00],
       [ -1.26418478e

当数据预处理完后，需要选择有意义的特征输入机器学习的算法和模型进行训练。通常从两个角度考虑：
+ 特征是否发散：如果一个特征不发散，例如方差接近于0.则说明样本在这个特征上**基本没有差异**。这个特征对于样本的区分没有什么作用。
+ 特征与目标的相关性：与目标相关性高的特征，应当优先选择。除方差法外，其他方法均从相关性考虑。

根据特征选择的形式将特征选择方法分为3种：
+ Fiter：过滤法。按照发散性或相关性对各个特征进行评分。设定阈值或者待选择阈值的个数，选择特征。
+ Wrapper：包装法。根据目标函数计算记过，每次选择若干特征，或者排除若干特征。
+ Embedded：集成法。先使用某些机器学习算法和模型进行训练，得到各个特征的权值系数，再根据系数从大到小选择特征。

#### Filter
##### 方差选择法
先计算各个特征的方法，然后根据阈值选择方差大于阈值的特征。使用`feature_selection`库的`VarianceThreshold`类来选择特征。

In [17]:
from sklearn.feature_selection import VarianceThreshold